In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline
pd.set_option('display.max_columns', None)

In [ ]:
#https://www.statsamerica.org/downloads/default.aspx         -population stats - u.s., state, county populaton
pop_data = pd.read_csv('data/population_data/Population_estimates_US_States_Counties.csv')

In [ ]:
pop_data.head(3)

In [ ]:
pop_data.info()

In [ ]:
pop_data_2019 = pop_data[pop_data['Year']==2019]
pop_data_2019

In [ ]:
#https://www.statsamerica.org/downloads/default.aspx           -population stats - u.s., state, county population change
pop_data2 = pd.read_csv('data/population_data/Components of Population Change - US, States, Counties.csv')

In [ ]:
pop_data2.head(3)

In [ ]:
pop_data2.info()

In [ ]:
pop_data2_2019 = pop_data2[pop_data2['Year']==2019]
pop_data2_2019

In [ ]:
#https://www.statsamerica.org/downloads/default.aspx         -population stats - u.s., state, county population by age
pop_data3 = pd.read_csv('data/population_data/Population by Age and Sex - US, States, Counties.csv')

In [ ]:
pop_data3.head(3)

In [ ]:
pop_data3.info()

In [ ]:
pop_data3_2019 = pop_data3[pop_data3['Year']==2019]
pop_data3_2019

In [ ]:
#https://www.statsamerica.org/downloads/default.aspx       - u.s. state, county employee wage info
pop_data4 = pd.read_csv('data/population_data/CEW - US, States, Counties - Total Ownership.csv')

In [ ]:
pop_data4.head(3)

In [ ]:
pop_data4.info()

In [ ]:
pop_data4_2019 = pop_data4[pop_data4['Year']==2019]
pop_data4_2019

In [ ]:
pop_data4_2019_totals = pop_data4_2019[pop_data4_2019['NAICS Description']=='Total']
pop_data4_2019_totals

In [ ]:
#https://ghdx.healthdata.org/record/ihme-data/united-states-life-expectancy-by-county-race-ethnicity-2000-2019
pop_data5 = pd.read_csv('data/population_data/IHME_USA_LE_COUNTY_RACE_ETHN_2000_2019_LT_2019_BOTH_Y2022M06D16.csv')

In [ ]:
pop_data5.head(3)

In [ ]:
pop_data5.info()

In [ ]:
#only finding totals which is cmbined totals for all races
pop_data5T = pop_data5[pop_data5['race_name']=='Total']

In [ ]:
#looking up life expectancy for <1 which is essentially life expectency at birth
pop_data5T1 = pop_data5T[pop_data5T['age_name']=='<1 year']

In [ ]:
# Split the fips column to create Statefips and Countyfips, replacing empty strings with 0
pop_data5T1['fips'] = pop_data5T1['fips'].astype(str)

pop_data5T1['Statefips'] = pop_data5T1['fips'].apply(lambda x: str(int(x.split('.')[0][:-3])) if '.' in x and x.split('.')[0][:-3] else '0')
pop_data5T1['Countyfips'] = pop_data5T1['fips'].apply(lambda x: str(int(x.split('.')[0][-3:])) if '.' in x and x.split('.')[0][-3:] else '0')

In [ ]:
#findind all cases where statefips=0 and county fips is != 0
mask = (pop_data5T1['Statefips'] == '0') & (pop_data5T1['Countyfips'] != '0')

# Update 'Statefips' with values from 'Countyfips' where the condition is true
pop_data5T1.loc[mask, 'Statefips'] = pop_data5T1.loc[mask, 'Countyfips']

# Update 'Countyfips' to '0' where the condition is true
pop_data5T1.loc[mask, 'Countyfips'] = '0'

In [ ]:
#convert both columns to int for merge
pop_data5T1['Statefips'] = pop_data5T1['Statefips'].astype(int)
pop_data5T1['Countyfips'] = pop_data5T1['Countyfips'].astype(int)

In [ ]:
#merge 1
pop_2019_merged = pd.merge(pop_data_2019, pop_data2_2019, left_on = ['IBRC_Geo_ID', 'Statefips', 'Countyfips',  'Description', 'Year'], 
                        right_on = ['IBRC_Geo_ID', 'Statefips', 'Countyfips',  'Description', 'Year'], how = 'inner')

In [ ]:
#merge 2
pop_2019_merged2 = pd.merge(pop_2019_merged, pop_data3_2019, left_on = ['IBRC_Geo_ID', 'Statefips', 'Countyfips', 'Description', 'Year'], 
                        right_on = ['IBRC_Geo_ID', 'Statefips', 'Countyfips', 'Description', 'Year'], how = 'inner')

In [ ]:
#merge 3
pop_2019_merged3 = pd.merge(pop_2019_merged2, pop_data4_2019_totals, left_on = [ 'Description', 'Year'], 
                        right_on = ['Description', 'Year'], how = 'inner')

In [ ]:
#merge 4
pop_all_data = pd.merge(pop_2019_merged3, pop_data5T1, left_on = [ 'Statefips_x', 'Countyfips_x'], 
                        right_on = ['Statefips', 'Countyfips'], how = 'inner')

In [ ]:
pop_all_data.head(3)

In [ ]:
pop_all_data = pop_all_data.drop(columns = ['Statefips_x',	'Countyfips_x', 'Count or Estimate', 'Residual', 'IBRC_GEO_ID',	'Statefips_y', 'Countyfips_y', 
                             'Ownership Code', 'Ownership Code Description', 'NAICS Code', 'NAICS Description', 'Disclosure', 'Summary Level',
                             'measure_id', 'measure_name', 'location_id', 'location_name', 'fips', 'race_id', 'race_name', 'sex_id', 'sex_name',
                             'age_group_id', 'age_name', 'year', 'metric_id', 'metric_name', 'upper', 'lower','Units' ])

In [ ]:
pop_all_data = pop_all_data.rename(columns={'val': 'Life Expectancy','Employment':'Employed Workers', 'Wages': 'Total Wages'}).round(2)

In [ ]:
pop_all_data = pop_all_data[['IBRC_Geo_ID', 'Statefips', 'Countyfips', 'Description', 'Year', 'Population', 'State or County Release', 'Births', 'Deaths',
                             'Life Expectancy', 'Net International Migration', 'Net Domestic Migration', 'Total Population', 'Population 0-4',
                             'Population 5-17', 'Population 18-24', 'Population 25-44', 'Population 45-64', 'Population 65+',
                             'Population Under 18', 'Population 18-54', 'Population 55+', 'Male Population', 'Female Population'
                             , 'Employed Workers', 'Total Wages', 'Average Wage', 'Average Weekly Wage']]


In [ ]:
pop_all_data

#https://taxfoundation.org/data/all/state/sales-tax-rates-2019/
state and local sales tax rates for 2019- requires webscrapping

#https://taxfoundation.org/data/all/state/state-individual-income-tax-rates-brackets-2019/
2019 state income taxes - copied to excel file as  data/tax_data/2019_state_income_tax_rates

#https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2019-zip-code-data-soi
individual income tax return data by state data/tax_data/2019_state_income_tax_returns

#https://www.census.gov/data/datasets/2019/econ/local/public-use-datasets.html
2019 state finance information revenue / expenditure data/tax_data/2019_state_finance_data

#https://www.census.gov/data/tables/2019/econ/qtax/historical.html
2019 state tax revenue, quarterly breakdown data/tax_data/2019_state_tax_revenue